In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Download packages

In [3]:
!pip uninstall -y tensorflow
!pip install -q tf-nightly-gpu
!pip install -q tensorflow-model-optimization==0.5
!pip install tensorflow==2.4.0

import tempfile
import os
import numpy as np

import tensorflow as tf

from tensorflow import keras

import tensorflow_model_optimization as tfmot

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 434.0MB 24kB/s 
     |████████████████████████████████| 6.0MB 49.9MB/s 
     |████████████████████████████████| 471kB 52.9MB/s 
     |████████████████████████████████| 4.0MB 52.5MB/s 
     |████████████████████████████████| 4.0MB 49.0MB/s 
     |████████████████████████████████| 3.9MB 49.2MB/s 
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
     |████████████████████████████████| 174kB 6.5MB/s 
     |████████████████████████████████| 394.7MB 41kB/s 
     |████████████████████████████████| 2.9MB 51.7MB/s 
     |████████████████████████████████| 3.8MB 51.3MB/s 
ERROR: tf-nightly-gpu 2.5.0.dev20210318 has requirement gast==0.4.0, but you'll have gast 0.3.3 which is incompatible.
ERROR: tf-nightly-gpu 2.5.0.dev20210318 has requirement grpcio~=1.34.0, but you'll have grpcio 1.32.0 which is incompatible.
ERROR: tf-nightly-gpu 2.5.0.dev20210318 has requirem

## Download dataset MNIST and train a model

In [4]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)


11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/4
1688/1688 [==============================] - 34s 3ms/step - loss: 0.5342 - accuracy: 0.8579 - val_loss: 0.1133 - val_accuracy: 0.9713
Epoch 2/4
1688/1688 [==============================] - 4s 2ms/step - loss: 0.1245 - accuracy: 0.9648 - val_loss: 0.0840 - val_accuracy: 0.9762
Epoch 3/4
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0812 - accuracy: 0.9764 - val_loss: 0.0686 - val_accuracy: 0.9820
Epoch 4/4
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0664 - accuracy: 0.9798 - val_loss: 0.0664 - val_accuracy: 0.9817


Evaluate baseline test accuracy and save the model for later usage.

In [5]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)


Baseline test accuracy: 0.9782999753952026
Saved baseline model to: /tmp/tmpkels4rlr.h5


##Fine-tune pre-trained model with pruning

In [6]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape  (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d ( (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


##Train and evaluate the model against baseline

In [14]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)


Epoch 1/2
422/422 [==============================] - 5s 7ms/step - loss: 0.0874 - accuracy: 0.9766 - val_loss: 0.1280 - val_accuracy: 0.9688
Epoch 2/2
422/422 [==============================] - 3s 6ms/step - loss: 0.1194 - accuracy: 0.9704 - val_loss: 0.0912 - val_accuracy: 0.9752


For this example, there is minimal loss in test accuracy after pruning, compared to the baseline

In [15]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)


Baseline test accuracy: 0.9782999753952026
Pruned test accuracy: 0.9714999794960022


##Create smaller models from pruning

In [16]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)


Saved pruned Keras model to: /tmp/tmp7w81ia2f.h5


In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)


INFO:tensorflow:Assets written to: /tmp/tmpp0fxo2mz/assets


INFO:tensorflow:Assets written to: /tmp/tmpp0fxo2mz/assets


Saved pruned TFLite model to: /tmp/tmpbx2ykjm7.tflite



Applying a standard compression algorithm is necessary since the serialized weight matrices are the same size as they were before pruning. However, pruning makes most of the weights zeros, which is added redundancy that algorithms can utilize to further compress the model.

In [18]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)


In [19]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))


Size of gzipped baseline Keras model: 78102.00 bytes
Size of gzipped pruned Keras model: 25677.00 bytes
Size of gzipped pruned TFlite model: 25010.00 bytes


## Excercice

What can we see in term of the size and the accuracy of models ? *

Try to create a smaller model from combining pruning and quantization

## Clean Up

Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)